# Gemma-2B Modelltraining in Google Colab Pro


In diesem Notebook werden wir ein leichtes Sprachmodell, Gemma-2B, in Google Colab Pro laden, es mit einem speziellen Datensatz trainieren und anschließend das trainierte Modell speichern. Dieses Notebook ist für die Arbeit mit beschränkten Ressourcen optimiert und nutzt Modelle, die effizient auf Google Colab ausgeführt werden können.


In [1]:

# Install necessary libraries
!pip install transformers datasets bitsandbytes huggingface_hub
!pip install ctransformers  # Für die Arbeit mit GGUF-Modellen


  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached tqdm-4.66.5-py3-none-any.whl.metadata (57 kB)
INFO: pip is looking at multiple versions of tokenizers to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of tokenizers to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.


ERROR: Invalid requirement: '#': Expected package name at the start of dependency specifier
    #
    ^


## Modell von Hugging Face laden

In [2]:

from huggingface_hub import snapshot_download

# Lade das Gemma-2B Modell herunter
model_id = "google/gemma-2b"
snapshot_download(repo_id=model_id, local_dir="gemma-2b")


Fetching 12 files:   0%|          | 0/12 [00:00<?, ?it/s]

GatedRepoError: 401 Client Error. (Request ID: Root=1-66d06ab8-07588401657b07a3538d9179;e8c91b55-8ae2-4d6b-8108-2ca0c00a97a3)

Cannot access gated repo for url https://huggingface.co/google/gemma-2b/resolve/68e273d91b1d6ea57c9e6024c4f887832f7b43fa/.gitattributes.
Access to model google/gemma-2b is restricted. You must be authenticated to access it.

## Konvertierung des Modells in GGUF-Format

In [ ]:

# Klonen des llama.cpp Repositories
!git clone https://github.com/ggerganov/llama.cpp.git

# Konvertiere das Modell in das GGUF-Format
!python llama.cpp/convert.py gemma-2b --outfile gemma-2b.gguf --outtype q4_K_M


## Datensatz vorbereiten und das Modell trainieren

In [ ]:

from transformers import AutoModelForCausalLM, AutoTokenizer, Trainer, TrainingArguments
from datasets import load_dataset

# Lade das Tokenizer und Modell
tokenizer = AutoTokenizer.from_pretrained("google/gemma-2b")
model = AutoModelForCausalLM.from_pretrained("google/gemma-2b")

# Lade und bereite den Datensatz vor
dataset = load_dataset("Abirate/english_quotes")
dataset = dataset.map(lambda samples: tokenizer(samples["quote"]), batched=True)

# Trainingsargumente definieren
training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=2,
    num_train_epochs=3,
    logging_dir='./logs',
)

# Trainer konfigurieren
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
)

# Training starten
trainer.train()


## Modell speichern

In [ ]:

from google.colab import drive
drive.mount('/content/drive')

# Speichere das trainierte Modell in Google Drive
model.save_pretrained("/content/drive/MyDrive/trained_model")
tokenizer.save_pretrained("/content/drive/MyDrive/trained_model")


## Modell auf Hugging Face hochladen

In [ ]:

from huggingface_hub import HfApi

# Lade das Modell auf Hugging Face hoch
api = HfApi()
api.upload_file(
    path_or_fileobj="gemma-2b.gguf",
    path_in_repo="gemma-2b.gguf",
    repo_id="your-username/gemma-2b"
)
